<a href="https://colab.research.google.com/github/lolobq/ECGR-5105-Intro_To_Machine_Learning/blob/main/Homework6/Homework6Problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Data

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Problem 2a

Create a fully connected Neural Network for all 10 classes in CIFAR-10 with only one hidden layer with a size of 512. Report your training time, training loss, and evaluation accuracy. Analyze your results in your report. Make sure to submit your code by providing the GitHub URL of your course repository for this course.

In [4]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the model
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 10)
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()  # Record the start time for training duration
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate training time
    end_time = time.time()
    training_time = end_time - start_time

    # Print training statistics
    if epoch % 10 == 0:
      print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

    # Print total training time
    total_end_time = time.time()
    total_training_time = total_end_time - total_start_time
    print(f'Total Training Time: {total_training_time:.2f} seconds')

    # Testing the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total

    # Print evaluation accuracy
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    # Set the model back to training mode
    model.train()

# After training is complete, print final evaluation accuracy
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 1.998261047903534, Training Time: 25.57 seconds
Total Training Time: 25.58 seconds
Test Accuracy: 34.42%
Total Training Time: 52.93 seconds
Test Accuracy: 37.20%
Total Training Time: 80.35 seconds
Test Accuracy: 38.52%
Total Training Time: 109.28 seconds
Test Accuracy: 39.32%
Total Training Time: 136.27 seconds
Test Accuracy: 39.65%
Total Training Time: 163.08 seconds
Test Accuracy: 40.14%
Total Training Time: 190.20 seconds
Test Accuracy: 40.34%
Total Training Time: 217.48 seconds
Test Accuracy: 40.81%
Total Training Time: 245.02 seconds
Test Accuracy: 41.28%
Total Training Time: 272.56 seconds
Test Accuracy: 41.55%
Epoch 11/300, Loss: 1.699005213082599, Training Time: 25.59 seconds
Total Training Time: 301.51 seconds
Test Accuracy: 41.53%
Total Training Time: 328.99 seconds
Test Accuracy: 42.08%
Total Training Time: 355.92 seconds
Test Accuracy: 42.11%


# Problem 2b

Extend your network with two more additional hidden layers, like the example we did in the lecture (pick the sizes of hidden layers properly). Train your network. Report your training time, loss, and evaluation accuracy after 300 epochs. Analyze your results in your report and compare your model size and accuracy over the baseline implementation in Problem 2.a. Do you see any over-fitting? Can you compare your model complexity against problem 2.a? Make sure to submit your code by providing the GitHub URL of your course repository for this course.

In [5]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the model with two additional hidden layers
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.ReLU(),
    nn.Linear(512, 256),  # Additional hidden layer 1
    nn.ReLU(),
    nn.Linear(256, 128),  # Additional hidden layer 2
    nn.ReLU(),
    nn.Linear(128, 10)
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()  # Record the start time for training duration
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate training time
    end_time = time.time()
    training_time = end_time - start_time

    # Print training statistics
    if epoch % 10 == 0:
      print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

    # Print total training time
    total_end_time = time.time()
    total_training_time = total_end_time - total_start_time
    print(f'Total Training Time: {total_training_time:.2f} seconds')

    # Testing the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total

    # Print evaluation accuracy
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    # Set the model back to training mode
    model.train()

# After training is complete, print final evaluation accuracy
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 2.2944025691512904, Training Time: 25.25 seconds
Total Training Time: 25.25 seconds
Test Accuracy: 17.27%
Total Training Time: 54.01 seconds
Test Accuracy: 21.41%
Total Training Time: 84.46 seconds
Test Accuracy: 24.19%
Total Training Time: 113.17 seconds
Test Accuracy: 26.59%
Total Training Time: 141.89 seconds
Test Accuracy: 28.35%
Total Training Time: 170.58 seconds
Test Accuracy: 29.68%
Total Training Time: 199.43 seconds
Test Accuracy: 31.21%
Total Training Time: 229.57 seconds
Test Accuracy: 32.62%
Total Training Time: 258.40 seconds
Test Accuracy: 34.15%
Total Training Time: 287.18 seconds
Test Accuracy: 35.73%
Epoch 11/300, Loss: 1.8177196999340106, Training Time: 25.27 seconds
Total Training Time: 315.90 seconds
Test Accuracy: 36.95%
Total Training Time: 344.62 seconds
Test Accuracy: 37.92%
Total Training Time: 373.33 seconds
Test Accuracy: 38.93